In [253]:
# Importing important libraries
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin, BaseEstimator
import pandas as pd
from sklearn.impute import SimpleImputer

In [254]:
# Loading the data
df = pd.read_excel('data/dataset_2023.xlsx')
df = df.drop('Unnamed: 0', axis=1)
df_test = df.iloc[:5,]
df_test

,brand_name,model_name,milage,first_registration,duration,gear,monthly_fee,emission_value,consumption,horsepower,kilowatts,fuel_type
0,Skoda,Octavia ŠKODA Combi Style TDI DSG,201 km,03/2023,48 Monat (anpassbar),Automatik,"574,01 €",119 g/km,"5,0 l/100 km",150 PS,110 kW,Diesel
1,Volkswagen,T-Cross VW Life TSI,201 km,03/2023,48 Monat (anpassbar),Manuelle Schaltung,"382,58 €",131 g/km,"6,0 l/100 km",95 PS,70 kW,Benzin
2,Seat,Ibiza Austria Edition,15.000 km,10/2022,48 Monat (anpassbar),Manuelle Schaltung,"239,62 €",120 g/km,"5,0 l/100 km",80 PS,59 kW,Benzin
3,Volkswagen,Polo VW,1 km,01/2023,48 Monat (anpassbar),Manuelle Schaltung,"309,11 €",127 g/km,"6,0 l/100 km",80 PS,59 kW,Benzin
4,Audi,A4 Avant 40 TDI quattro S line,105.301 km,12/2019,48 Monat (anpassbar),Automatik,"587,75 €",138 g/km,"5,0 l/100 km",190 PS,140 kW,Diesel


In [255]:
def replaceComma(text):
    if "." in str(text): 
        return str("".join(str(text).split("."))).replace(",",".")
    else:
        return str(text).replace(",",".")

def removeLetters(text):
    return "".join(c for c in text if c.isdigit() or c == ".")

def removeThousandPoint(text):
    return str(text).replace(".","")

from datetime import datetime
from dateutil.relativedelta import relativedelta

def calculate_age(registration_date):
    if registration_date == "Neuwagen":
        return 0
    else:
        today = datetime.now()
        date = datetime.strptime(registration_date, "%m/%Y")
        delta = relativedelta(today, date)
        return (delta.years * 12) + delta.months
    
def getConsumption(consumption):
    return str(consumption).split()[0]    

In [256]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.base import BaseEstimator, TransformerMixin

class AgeTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        def calculate_age(registration_date):
            if registration_date == "Neuwagen":
                return 0
            else:
                today = datetime.now()
                date = datetime.strptime(registration_date, "%m/%Y")
                delta = relativedelta(today, date)
                return (delta.years * 12) + delta.months
        
        X_copy = X.copy()
        X_copy["first_registration"] = X_copy["first_registration"].apply(calculate_age) 
        X_copy["first_registration"] = X_copy["first_registration"].replace('', np.nan)
        X_copy["first_registration"] = X_copy["first_registration"].replace('nan', np.nan)
        X_copy["first_registration"] = pd.to_numeric(X_copy["first_registration"], errors='coerce')

        return X_copy
    
class getConsumption(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        def getConsumption(consumption):
            return str(consumption).split()[0]
        
        X_copy = X.copy()
        X_copy['consumption'] = X_copy['consumption'].apply(getConsumption)
        X_copy["consumption"] = X_copy["consumption"].apply(replaceComma) 
        X_copy["consumption"] = X_copy["consumption"].replace('', np.nan)
        X_copy["consumption"] = X_copy["consumption"].replace('nan', np.nan)
        X_copy["consumption"] = pd.to_numeric(X_copy["consumption"], errors='coerce')

        return X_copy

class transformMilage(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):

        X_copy = X.copy()
        X_copy["milage"] = X_copy["milage"].apply(replaceComma)
        X_copy["milage"] = X_copy["milage"].apply(removeLetters)   
        X_copy["milage"] = X_copy["milage"].replace('', np.nan)
        X_copy["milage"] = X_copy["milage"].replace('nan', np.nan)
        X_copy["milage"] = pd.to_numeric(X_copy["milage"], errors='coerce')

        return X_copy
    
class transformDuration (BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):

        X_copy = X.copy()
        X_copy["duration"] = X_copy["duration"].apply(replaceComma)
        X_copy["duration"] = X_copy["duration"].apply(removeLetters)   
        X_copy["duration"] = X_copy["duration"].replace('', np.nan)
        X_copy["duration"] = X_copy["duration"].replace('nan', np.nan)
        X_copy["duration"] = pd.to_numeric(X_copy["duration"], errors='coerce')

        return X_copy    
    
class transformFee (BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):

        X_copy = X.copy()
        X_copy["monthly_fee"] = X_copy["monthly_fee"].apply(replaceComma)
        X_copy["monthly_fee"] = X_copy["monthly_fee"].apply(removeLetters)    
        X_copy["monthly_fee"] = X_copy["monthly_fee"].replace('', np.nan)
        X_copy["monthly_fee"] = X_copy["monthly_fee"].replace('nan', np.nan)
        X_copy["monthly_fee"] = pd.to_numeric(X_copy["monthly_fee"], errors='coerce')

        return X_copy 
    
class transformEmission (BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):

        X_copy = X.copy()
        X_copy["emission_value"] = X_copy["emission_value"].apply(replaceComma)
        X_copy["emission_value"] = X_copy["emission_value"].apply(removeLetters)   
        X_copy["emission_value"] = X_copy["emission_value"].replace('', np.nan)
        X_copy["emission_value"] = X_copy["emission_value"].replace('nan', np.nan)
        X_copy["emission_value"] = pd.to_numeric(X_copy["emission_value"], errors='coerce')
        
        return X_copy 
    
class transformHorsepower (BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):

        X_copy = X.copy()
        X_copy["horsepower"] = X_copy["horsepower"].apply(replaceComma)
        X_copy["horsepower"] = X_copy["horsepower"].apply(removeLetters)
        X_copy["horsepower"] = X_copy["horsepower"].replace('', np.nan)
        X_copy["horsepower"] = X_copy["horsepower"].replace('nan', np.nan)   
        X_copy["horsepower"] = pd.to_numeric(X_copy["horsepower"], errors='coerce')
     
        return X_copy 
    
class transformKilowatts (BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):

        X_copy = X.copy()
        X_copy["kilowatts"] = X_copy["kilowatts"].apply(replaceComma)
        X_copy["kilowatts"] = X_copy["kilowatts"].apply(removeLetters) 
        X_copy["kilowatts"] = X_copy["kilowatts"].replace('', np.nan)
        X_copy["kilowatts"] = X_copy["kilowatts"].replace('nan', np.nan) 
        X_copy["kilowatts"] = pd.to_numeric(X_copy["kilowatts"], errors='coerce')
  
        return X_copy 
    
    
class passThrough(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X
    





In [257]:


class PandasTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, transformer):
        self.transformer = transformer
        
    def fit(self, X, y=None):
        self.transformer.fit(X, y)
        return self
        
    def transform(self, X):
        transformed = self.transformer.transform(X)
        return pd.DataFrame(transformed, columns=X.columns)


In [258]:
df_test

,brand_name,model_name,milage,first_registration,duration,gear,monthly_fee,emission_value,consumption,horsepower,kilowatts,fuel_type
0,Skoda,Octavia ŠKODA Combi Style TDI DSG,201 km,03/2023,48 Monat (anpassbar),Automatik,"574,01 €",119 g/km,"5,0 l/100 km",150 PS,110 kW,Diesel
1,Volkswagen,T-Cross VW Life TSI,201 km,03/2023,48 Monat (anpassbar),Manuelle Schaltung,"382,58 €",131 g/km,"6,0 l/100 km",95 PS,70 kW,Benzin
2,Seat,Ibiza Austria Edition,15.000 km,10/2022,48 Monat (anpassbar),Manuelle Schaltung,"239,62 €",120 g/km,"5,0 l/100 km",80 PS,59 kW,Benzin
3,Volkswagen,Polo VW,1 km,01/2023,48 Monat (anpassbar),Manuelle Schaltung,"309,11 €",127 g/km,"6,0 l/100 km",80 PS,59 kW,Benzin
4,Audi,A4 Avant 40 TDI quattro S line,105.301 km,12/2019,48 Monat (anpassbar),Automatik,"587,75 €",138 g/km,"5,0 l/100 km",190 PS,140 kW,Diesel


In [259]:
# define the pipeline steps
age_transformer = AgeTransformer()
get_consumption = getConsumption()
pass_through = passThrough()
transform_milage = transformMilage()
transform_duration = transformDuration()
transform_fee = transformFee()
transfrom_emission = transformEmission()
transform_horsepower = transformHorsepower()
transform_kilowatts = transformKilowatts()
imputer_numeric = SimpleImputer(missing_values = np.nan, strategy="mean")
imputer_cat = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
#imputer_cat_NA = SimpleImputer(missing_values = np.nan, strategy='most_frequent')
#imputer_numeric_NA = SimpleImputer(missing_values = np.nan, strategy='most_frequent')

column_transformer = ColumnTransformer([
    ("brand_passthrough", pass_through, ["brand_name"]),
    ("model_name", pass_through, ["model_name"]),
    ("milage", transform_milage, ["milage"]),
    ("age_transformer", age_transformer, ["first_registration"]),
    ("duration", transform_duration, ["duration"]),
    ("gear", pass_through, ["gear"]),
    ("monthly_fee", transform_fee, ["monthly_fee"]),
    ("emission_value", transfrom_emission, ["emission_value"]),
    ("getConsumption", get_consumption, ["consumption"]), 
    ("horsepower", transform_horsepower, ["horsepower"]),
    ("kilowatts", transform_kilowatts, ["kilowatts"]),
    ("fuel_type", pass_through, ["fuel_type"]),
  #  ('cat_imputer', imputer_cat, ['brand_name', "model_name", "gear", "fuel_type"]),
   # ('num_imputer', imputer_numeric, ['milage', 'first_registration', "duration", "monthly_fee", "emission_value", "consumption", "horsepower", "kilowatts"])
    
], remainder="passthrough")



# wrap the column transformer in the PandasTransformer
pipeline = Pipeline([
    ("column_transformer", PandasTransformer(column_transformer)),
   # ("imputer_numeric", PandasTransformer(imputer_numeric)),

])

# fit and transform the data
transformed_df = pipeline.fit_transform(df)


In [260]:
transformed_df

,brand_name,model_name,milage,first_registration,duration,gear,monthly_fee,emission_value,consumption,horsepower,kilowatts,fuel_type
0,Skoda,Octavia ŠKODA Combi Style TDI DSG,201,2,48,Automatik,574.01,119.0,5.0,150,110,Diesel
1,Volkswagen,T-Cross VW Life TSI,201,2,48,Manuelle Schaltung,382.58,131.0,6.0,95,70,Benzin
2,Seat,Ibiza Austria Edition,15000,7,48,Manuelle Schaltung,239.62,120.0,5.0,80,59,Benzin
3,Volkswagen,Polo VW,1,4,48,Manuelle Schaltung,309.11,127.0,6.0,80,59,Benzin
4,Audi,A4 Avant 40 TDI quattro S line,105301,41,48,Automatik,587.75,138.0,5.0,190,140,Diesel
...,...,...,...,...,...,...,...,...,...,...,...,...
19053,Seat,Ateca FR 2.0 TDI DSG 4Drive,201,4,48,Automatik,692.03,146.0,6.0,150,110,Diesel
19054,Skoda,Octavia ŠKODA Combi Style TDI DSG,201,2,48,Automatik,574.01,187.0,8.0,150,110,Diesel
19055,Audi,A4 Avant 40 TDI quattro S line,105301,41,48,Automatik,587.75,143.0,6.0,190,140,Diesel
19056,Volkswagen,Polo VW,18903,35,48,Manuelle Schaltung,256.33,40.0,2.0,80,59,Benzin


In [261]:
transformed_df["duration"].mean()

46.870395634379264